# Домашно 2

Трениране и подобряване на модели

Предайте същата тетрадка като тази в заданието с нанесените от вас промени. Кръстете файла с факултетния си номер.

# Задача 1

В заданието  имате данните отностно сърдечни заболявания, които са взети от [тук](https://archive.ics.uci.edu/dataset/222/bank+marketing).
Данните, които трябва да прочетете, са файлът `bank.csv`

1.1 Прочетете набора от данни с помощта на `pandas`.

1.2 След което разбийте данните на атрибути и целеви атрибут. в случая целевия атрибут е y.

1.3 Подгответе категорийните променливи, така че да могат да бъдат обработвани като индикатори.

1.4 Разбийте данните на тренировъчно и тестово множество (като 30% от данните да са в тестовото множество). 

Преди да ги разбиете вижте какъв е баланса между двата класа и помислете как трябва да ги разбиете(hint: stratification) 

1.5 Скалирайте данните така, че да имате средно 0 и стандартно отклонение 1.


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('bank.csv', sep=';')

X = data.drop('y', axis=1) 
y = data['y'].map({'yes': 1, 'no': 0})

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Задача 2 Дървета

Използвайте крос-валидация с 5 гънки(5-fold cross validation), за да обучите и оптимизирате хиперпараметрите за дърво на решенията.

Помислете внимателно как трябва да разбиете гънките при толкова небалансирани класове(hint: stratified k-fold cross-validation).

Помислете как да оценявате модела не забравяйте, че точността не работи в този случай, защото данните са силно небалансирани. Имаме нужда от нещо, което съчетава прецизност и обхват(възвръщаемост).

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `max_depth`
* `min_samples_leaf`
* `max_leaf_nodes`

Вижте как се представя алгоритъма върху тренировачните данни и кои са най-добрите хипер параметри.

In [34]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, recall_score

tree_classifier = DecisionTreeClassifier()

param_grid = {
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [2, 4, 6],
    'max_leaf_nodes': [None, 5, 10]
}

scorer = make_scorer(recall_score)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=tree_classifier, param_grid=param_grid, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_params_dtc = grid_search.best_params_
best_score = grid_search.best_score_

print("Best hyperparameters:", best_params_dtc)
print("Best score (recall):", best_score)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best hyperparameters: {'max_depth': 5, 'max_leaf_nodes': 10, 'min_samples_leaf': 6}
Best score (recall): 0.3808219178082191


# Задача 3 Случайна гора

Аналогично на предишната задача но този път със случайна гора

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `n_estimators`
* `max_depth`


In [36]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15]
}

scorer = make_scorer(recall_score)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_params_rf = grid_search.best_params_
best_score = grid_search.best_score_

print("Best hyperparameters:", best_params_rf)
print("Best score (recall):", best_score)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best hyperparameters: {'max_depth': 15, 'n_estimators': 50}
Best score (recall): 0.29315068493150687


# Задача 4 SVM

Аналогично на предишната задача но този път със svm

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `C`
* `gamma`


In [38]:
from sklearn.svm import SVC

svm_classifier = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01, 0.001]
}

scorer = make_scorer(recall_score)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_params_svm = grid_search.best_params_
best_score = grid_search.best_score_

print("Best hyperparameters:", best_params_svm)
print("Best score (recall):", best_score)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best hyperparameters: {'C': 10, 'gamma': 0.01}
Best score (recall): 0.37808219178082186


# Задача 5 

Повторете задачи 2-4 като изполвате резултатите получени до сега за да избрете нови стойности на същите хипер параметри за които да правите нов grid search.


In [39]:
# Decision Tree Classifier:

param_grid_dtc = {
    'max_depth': [best_params_dtc['max_depth'] - 1, best_params_dtc['max_depth'], best_params_dtc['max_depth'] + 1],
    'min_samples_leaf': [best_params_dtc['min_samples_leaf'] - 1, best_params_dtc['min_samples_leaf'], best_params_dtc['min_samples_leaf'] + 1],
    'max_leaf_nodes': [best_params_dtc['max_leaf_nodes'], best_params_dtc['max_leaf_nodes'] + 5, None]
}

grid_search_dt = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid_dtc, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search_dt.fit(X_train_scaled, y_train)

best_params_dtc_new = grid_search_dt.best_params_
best_score_dt_new = grid_search_dt.best_score_

print("Best hyperparameters for decision tree:", best_params_dtc_new)
print("Best score (recall) for decision Tree:", best_score_dt_new)

# Random Forest Classifier:

param_grid_rf = {
    'n_estimators': [best_params_rf['n_estimators'] - 25, best_params_rf['n_estimators'], best_params_rf['n_estimators'] + 25],
    'max_depth': [best_params_rf['max_depth'] - 2, best_params_rf['max_depth'], best_params_rf['max_depth'] + 2]
}

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid_rf, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search_rf.fit(X_train_scaled, y_train)

best_params_rf_new = grid_search_rf.best_params_
best_score_rf_new = grid_search_rf.best_score_

print("Best hyperparameters for random forest:", best_params_rf_new)
print("Best score (recall) for random forest:", best_score_rf_new)

# SVM:

best_params_svm = {'C': 10, 'gamma': 0.01}

param_grid_svm = {
    'C': [best_params_svm['C'] - 2, best_params_svm['C'], best_params_svm['C'] + 2],
    'gamma': [best_params_svm['gamma'] / 2, best_params_svm['gamma'], best_params_svm['gamma'] * 2]
}

grid_search_svm = GridSearchCV(estimator=SVC(), param_grid=param_grid_svm, scoring=scorer, cv=cv, verbose=1, n_jobs=-1)
grid_search_svm.fit(X_train_scaled, y_train)

best_params_svm_new = grid_search_svm.best_params_
best_score_svm_new = grid_search_svm.best_score_

print("Best hyperparameters for SVM:", best_params_svm_new)
print("Best score (recall) for SVM:", best_score_svm_new)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best hyperparameters for decision tree: {'max_depth': 6, 'max_leaf_nodes': 15, 'min_samples_leaf': 5}
Best score (recall) for decision Tree: 0.4575342465753424
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best hyperparameters for random forest: {'max_depth': 17, 'n_estimators': 50}
Best score (recall) for random forest: 0.2931506849315068
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Best hyperparameters for SVM: {'C': 10, 'gamma': 0.02}
Best score (recall) for SVM: 0.410958904109589


# Задача 6

Сравнете как се предстявят всички модели и изберете кой е най-добрия


In [41]:
from sklearn.metrics import recall_score

best_dt = DecisionTreeClassifier(max_depth=best_params_dtc['max_depth'], min_samples_leaf=best_params_dtc['min_samples_leaf'], max_leaf_nodes=best_params_dtc['max_leaf_nodes'])
best_rf = RandomForestClassifier(n_estimators=best_params_rf['n_estimators'], max_depth=best_params_rf['max_depth'])
best_svm = SVC(C=best_params_svm['C'], gamma=best_params_svm['gamma'])

best_dt.fit(X_train_scaled, y_train)
best_rf.fit(X_train_scaled, y_train)
best_svm.fit(X_train_scaled, y_train)

y_pred_dt = best_dt.predict(X_test_scaled)
y_pred_rf = best_rf.predict(X_test_scaled)
y_pred_svm = best_svm.predict(X_test_scaled)

recall_dt = recall_score(y_test, y_pred_dt)
recall_rf = recall_score(y_test, y_pred_rf)
recall_svm = recall_score(y_test, y_pred_svm)

print(f"Decision tree recall: {recall_dt:.4f}")
print(f"Random forest recall: {recall_rf:.4f}")
print(f"SVM recall: {recall_svm:.4f}")

best_model = max([(recall_dt, 'Decision Tree'), (recall_rf, 'Random Forest'), (recall_svm, 'SVM')])

print(f"\nThe best model is: {best_model[1]} with a recall of {best_model[0]:.4f}")


Decision tree recall: 0.3462
Random forest recall: 0.2179
SVM recall: 0.3333

The best model is: Decision Tree with a recall of 0.3462
